In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [41]:
# Read in the data
data = pd.read_csv("/home/ricardo/Github/trump_taiwan_tweets_analysis/datasets/original_tweet_text_merged_tab.csv", sep = "\t")

# drop duplicates rows
# remove duplicates rows
print('Before:', len(data))
data.drop_duplicates(inplace=True)
print('After: ', len(data))

# remeber to remove the row after checking the code
# data = data.iloc[:50, :]

is_retweet_list = []
retweet_account_list = []
tweet_text_list = []
original_text_list = []
retweet_text_list = []

for i, j in enumerate(tqdm(data.loc[:, 'Tweet'])):
    tweet_text_list.append(j)
    tweet_split_list = str(j).split(" ")
    # check if the tweet is retweeted
    if "RT" in tweet_split_list:
        is_retweet_list.append(1)
        # catch the index of RT
        RT_index = tweet_split_list.index('RT')
        # original text
        original_text = ""
        for a, b in enumerate(tweet_split_list[:RT_index]):
            original_text += " " + b
        original_text_list.append(original_text)
        # Retweeted Text
        retweeted_text = ""
        account_grep = False
        for a, b in enumerate(tweet_split_list[RT_index:]):
            if len(str(b)) >= 1:
                # grep account
                if str(b)[0] == "@" and account_grep == False:
                    account_grep = True
                    # eliminate ":" synbol
                    if str(b)[-1] == ":":
                        retweet_account_list.append(b[1:-1])
                    else:
                        retweet_account_list.append(b[1:])
                # do not append 'RT' text
                elif b == "RT":
                    pass
                else:
                    retweeted_text += " " + str(b) 
        if account_grep == False:
            retweet_account_list.append("")
        retweet_text_list.append(retweeted_text)
    else:
        is_retweet_list.append(0)
        retweet_account_list.append("")
        original_text_list.append(j)
        retweet_text_list.append("")

print(len(is_retweet_list))
print(len(retweet_account_list))
print(len(tweet_text_list))
print(len(original_text_list))
print(len(retweet_text_list))
        

output_data = pd.DataFrame({
    'is_RT':is_retweet_list,
    'retweet_from':retweet_account_list,
    'whole_tweet_text':tweet_text_list,
    'original_text':original_text_list,
    'retweet_text':retweet_text_list
})
        
output_data.to_csv("~/Github/trump_taiwan_tweets_analysis/datasets/original_splitted.csv", 
                   sep='\t', index=None)

100%|██████████| 31316/31316 [00:00<00:00, 379814.08it/s]

Before: 61989
After:  31316
31316
31316
31316
31316
31316


In [89]:
# consider original text and retweet data sperately
output_data = output_data.sort_values(by=['is_RT'], ascending=False)
print(len(output_data))

original_text_data = output_data[output_data['is_RT'] == 0]
print(len(original_text_data))
retweet_text_data = output_data[output_data['is_RT'] == 1]
print(len(retweet_text_data), '\n')

# remove duplicates of both dataframe
original_text_data = original_text_data.drop_duplicates(subset='original_text')
original_text_data.sort_values(by=['original_text'])
print(len(original_text_data))
retweet_text_data = retweet_text_data.drop_duplicates(subset='retweet_text')
retweet_text_data.sort_values(by=['retweet_text'])
print(len(retweet_text_data), '\n')

# man defined duplicates search, elastic similarity threshold = 0.4
retweet_text_data.to_csv("~/Github/trump_taiwan_tweets_analysis/datasets/retweet_text_without.csv", 
                   sep='\t', index=None)
for i, j in enumerate(retweet_text_data.iloc[:, 4]):
    content_list = j.split(" ")
    for a, b in enumerate(retweet_text_data.iloc[(i+1):, 4]):
        tocheck_content_list = b.split(" ")
        # checking procedure
        similar_ratio = (len(content_list + tocheck_content_list) - len(list(set(content_list + tocheck_content_list)))) / len(content_list + tocheck_content_list) 
        if similar_ratio >= 0.40:
#             print(similar_ratio, a, b)
            retweet_text_data = retweet_text_data[retweet_text_data['retweet_text'] != b]
print(len(retweet_text_data))
            

# export the datas
original_text_data.to_csv("~/Github/trump_taiwan_tweets_analysis/datasets/original_text.csv",
                          sep='\t', index=None)
retweet_text_data.to_csv("~/Github/trump_taiwan_tweets_analysis/datasets/retweet_text.csv", 
                   sep='\t', index=None)

31316
30567
749 

30567
741 

607
